In [1]:
import os, sys, copy, itertools
import numpy as np
from bionlp.util import fs
sys.path.append(os.path.abspath("C:\\Users\\yans2\\source\\py\\evntextrc\\bin"))
import bnlpst

YEAR = '2013'
TASK = 'bb2'
bnlpst.DATA_PATH = 'C:\\Users\\yans2\\data\\bioevent\\bnlpst'

In [2]:
spdr = bnlpst
train_docids, dev_docids, test_docids = spdr.get_docid(dataset='train', source=YEAR, task=TASK), spdr.get_docid(dataset='dev', source=YEAR, task=TASK), spdr.get_docid(dataset='test', source=YEAR, task=TASK)
train_raw_data = {
    'docids':train_docids,
    'corpus':spdr.get_corpus(train_docids, dataset='train', source=YEAR, task=TASK),
    'preprcs':spdr.get_preprcs(train_docids, dataset='train', source=YEAR, task=TASK, method='spacy'),
    'evnts':spdr.get_evnts(train_docids, dataset='train', source=YEAR, task=TASK)
}
dev_raw_data = {
    'docids':dev_docids,
    'corpus':spdr.get_corpus(dev_docids, dataset='dev', source=YEAR, task=TASK),
    'preprcs':spdr.get_preprcs(dev_docids, dataset='dev', source=YEAR, task=TASK, method='spacy'),
    'evnts':spdr.get_evnts(dev_docids, dataset='dev', source=YEAR, task=TASK)
}
test_raw_data = {
    'docids':test_docids,
    'corpus':spdr.get_corpus(test_docids, dataset='test', source=YEAR, task=TASK),
    'preprcs':spdr.get_preprcs(test_docids, dataset='test', source=YEAR, task=TASK, method='spacy'),
    'evnts':[[]] * len(test_docids)
}

# 1st Version

In [3]:
ds_name_prefix = 'bnlpst-%s-%s' % (YEAR, TASK)
for ds_name, raw_data in zip(['%s-train' % ds_name_prefix, '%s-dev' % ds_name_prefix, '%s-test' % ds_name_prefix], [train_raw_data, dev_raw_data, test_raw_data]):
    total_new_records = []
    for docid, preprc, txt, evnts in zip(raw_data['docids'], raw_data['preprcs'], raw_data['corpus'], raw_data['evnts']):
        new_records = []
        txt = txt.strip('\n').replace('\n', ' ')
        sent_bndry, words, annots, gddfs, coref = preprc
        if (len(evnts) == 0): # Testing set
            for locs, tp in zip(annots['loc'], annots['type']):
                for loc in locs:
                    newtxt.extend([txt[offset:loc[0]], '@%s$' % tp.upper()])
                    offset = loc[1]
            newtxt.append(txt[offset:])
            new_records.append('%s\t%s' % (docid, ''.join(newtxt)))
        else: # Training and dev set
            for eid, oprnds, evnt in zip(evnts['id'], evnts['oprnds'], evnts['type']):
                oprids = sorted([annots['id'].index(oprnd) for oprnd in oprnds])
                offset, newtxt = 0, []
                for locs, tp in zip([annots['loc'][x] for x in oprids], [annots['type'][x] for x in oprids]):
                    loc = locs[0] if (len(locs) == 1) else (np.min(locs), np.max(locs))
                    newtxt.extend([txt[offset:loc[0]], '@%s$' % tp.upper()])
                    offset = loc[1]
                newtxt.append(txt[offset:])
                new_records.append('%s-%s\t%s\t%s' % (docid, eid, ''.join(newtxt), evnt))
        total_new_records.extend(new_records)
    fs.write_file('\n'.join(total_new_records), ds_name)

# 2nd Version

In [8]:
ds_name_prefix = 'bnlpst-%s-%s' % (YEAR, TASK)
for ds_name, raw_data in zip(['%s-train.tsv' % ds_name_prefix, '%s-dev.tsv' % ds_name_prefix, '%s-test.tsv' % ds_name_prefix], [train_raw_data, dev_raw_data, test_raw_data]):
    total_new_records = []
    for docid, preprc, txt, evnts in zip(raw_data['docids'], raw_data['preprcs'], raw_data['corpus'], raw_data['evnts']):
        new_records = []
        txt = txt.rstrip('\n').replace('\n', ' ')
        sent_bndry, words, annots, gddfs, coref = preprc
        comb_pairs = ['-'.join(map(str, sorted(x))) for x in itertools.combinations(range(len(annots['id'])), 2)]
        if (len(evnts) == 0): # Testing set
            pass
#             for locs, tp in zip(annots['loc'], annots['type']):
#                 for loc in locs:
#                     newtxt.extend([txt[offset:loc[0]], '@%s$' % tp.upper()])
#                     offset = loc[1]
#             newtxt.append(txt[offset:])
#             new_records.append('%s\t%s' % (docid, ''.join(newtxt)))
        else: # Training and dev set
            for eid, oprnds, evnt in zip(evnts['id'], evnts['oprnds'], evnts['type']):
                oprids = sorted([annots['id'].index(oprnd) for oprnd in oprnds])
                offset, newtxt = 0, []
                for locs, tp in zip([annots['loc'][x] for x in oprids], [annots['type'][x] for x in oprids]):
                    loc = locs[0] if (len(locs) == 1) else (np.min(locs), np.max(locs))
                    newtxt.extend([txt[offset:max(loc[0], offset)], '@%s$' % tp.upper()])
                    offset = max(loc[1], offset)
                newtxt.append(txt[offset:])
                new_records.append('%s-%s\t%s\t%s' % (docid, eid, ''.join(newtxt), evnt))
                try:
                    comb_pairs.remove('-'.join(map(str, oprids)))
                except ValueError as e:
                    pass
        # Unannotated or Negative samples
        eid_offset = 1 if (len(evnts) == 0 or len(evnts['id']) == 0) else int(evnts['id'][-1].strip('ER')) + 1
        for eid, oprids_str in zip(['E%i'%x for x in range(eid_offset, eid_offset + len(comb_pairs))], comb_pairs):
            oprids = map(int, oprids_str.split('-'))
            offset, newtxt = 0, []
            for locs, tp in zip([annots['loc'][x] for x in oprids], [annots['type'][x] for x in oprids]):
                loc = locs[0] if (len(locs) == 1) else (np.min(locs), np.max(locs))
                newtxt.extend([txt[offset:loc[0]], '@%s$' % tp.upper()])
                offset = loc[1]
            newtxt.append(txt[offset:])
            new_records.append('%s\t%s' % (docid, ''.join(newtxt)) if (len(evnts) == 0) else '%s-%s\t%s\t%s' % (docid, eid, ''.join(newtxt), 'False'))
        total_new_records.extend(new_records)
    fs.write_file('\n'.join(total_new_records), ds_name)